

### Gridap day at "Groupe Calcul"

# Exercise 0


The following parallel code computes the Poisson equation with Dirichlet boundary conditions on the unit cube. It measures the time of the main phases of the algorithm and saves the results to a file. `n` is the number of cells in each direction. `np` is the number of parts in each direction. `nruns` is the number of times the algorithm is run. We need to run several times to obtain reliable time measures.

1. Run the code with  `n=(50,50,50)` for `np=(1,1,1)`, `np=(2,1,1)` and `np=(2,2,1)`.
To tun the code with more than 1 MPI rank, you need to export it into a `.jl` file and launch it as explained here : https://juliaparallel.org/MPI.jl/stable/usage/

2. Compute the speed-up of the main phases of the algorithm from the generated result files. For each combination of `n` and `np`, take the minimum time over the different runs and the maximum over MPI ranks. Remeber that `speed_up = time_for_1_proc / time_for_n_procs`.


In [ ]:
using Gridap
using GridapDistributed
using GridapPETSc
using PartitionedArrays
using FileIO

In [ ]:
function main(parts,cells,i,options)
    timer = PTimer(parts)
    u(x) = sum(x)
    f(x) = -Δ(u,x)
    g = u
    k = 1
    domain = (0,1,0,1,0,1)
    tic!(timer,barrier=true)
    model = CartesianDiscreteModel(parts,domain,cells)
    toc!(timer,"mesh")
    tic!(timer,barrier=true)
    reffe = ReferenceFE(lagrangian,Float64,k)
    Vh = TestFESpace(model,reffe,dirichlet_tags="boundary")
    Uh = TrialFESpace(Vh,g)
    toc!(timer,"space")
    tic!(timer,barrier=true)
    Ω = Interior(model)
    dΩ = Measure(Ω,2*k)
    a(u,v) = ∫(∇(u)⋅∇(v))dΩ
    l(v) = ∫(v*f)dΩ
    op = AffineFEOperator(a,l,Uh,Vh)
    toc!(timer,"assembly")
    uh = GridapPETSc.with(args=split(options)) do
        solver = PETScLinearSolver()
        tic!(timer,barrier=true)
        uh = solve(solver,op)
        toc!(timer,"solver")
        uh
    end
    tic!(timer,barrier=true)
    e = u - uh
    contrib_h1 = ∫(∇(e)⋅∇(e) + e*e )dΩ
    h1 = sqrt(sum(contrib_h1))
    toc!(timer,"error")
    display(timer)
    map_main(timer.data) do data
        ns = join(map(c->"_"*string(c),cells))
        nps = join(map(c->"_"*string(c),size(parts)))
        name = "results_run$(i)$(ns)$(nps).jld2"
        output = Dict{String,Any}(data)
        output["h1"] = h1
        save(name,output)
    end
    nothing
end

In [ ]:
backend = MPIBackend()
n = (50,50,50)
np = (1,1,1)
nruns = 3
options = "-pc_type gamg -ksp_type cg -ksp_converged_reason -ksp_rtol 1.0e-12"
prun(backend,np) do parts
    for i in 1:nruns
        main(parts,n,i,options)
    end
end